# TIME SERIES FORECASTING

***Hey so first things first, Thankyou for opening my kernel. Disclaimer : I am still learning but I have tried to jot down whatever I have understood in this kernel. Ps - Tag along till the end, you'll surely enjoy my commentary if not the code AND please upvote if you like this!!!***

 I will take you through time series data its basically a collection of data points over a time interval.

Steps : 
1. Loading and Indexing the Dataset
2. Checking if the data is stationary or not ( It should have constant Mean - Variance - Std)
3. Converting data into stationary 
4. Creating the ARIMA model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Loading and Reading the Dataset

In [ ]:
df = pd.read_csv('../input/air-passengers/AirPassengers.csv')

In [ ]:
df.head()

The dataset is time series having columns Months and No. of Passengers. Now we will play with the basic pandas function to know more about the dataset.

In [ ]:
df.shape

HURRAY!! No missing values is just the best thing which happened to me today

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
from datetime import datetime

Converting date string to datetime data type

In [ ]:
df['Month']=pd.to_datetime(df['Month'],infer_datetime_format=True)


As you can see, I have changed the data type now

In [ ]:
df.info()

Setting the index as date so that we can access and traverse values easily, do a substring search and brownie points to the fact that it can help us to visualise easily.

In [ ]:
index = df.set_index('Month',inplace=False)

In [ ]:
index.head()

Visualising the Passengers

In [ ]:
index.plot()

Typing "#Passengers" seems really annoying so I'll just rename it, Hehe.

In [ ]:
index.rename(columns={'#Passengers':'Passengers'},inplace=True)

# Checking if the data is stationary

So basically you can adopt any of these ways
-> plot the rooling mean or rolling std to check if the change in mean is constant or not 
-> ADCF Test
-> KPSS Test

For a better explanation and your clarity, I will skim you through each of them..

In [ ]:
simple_ma = index['Passengers'].rolling(window=12).mean()
simple_std = index['Passengers'].rolling(window=12).std()

In [ ]:
plt.title('Data vs Mean vs Std')
plt.plot(index,color='yellow')
plt.plot(simple_ma, color='blue')
plt.plot(simple_std,color='red')

Trend in the dataset can evidently be seen

Conducting the ADCF ie Augmented Dickey Fuller Test. Having the Null Hypothesis to be Time Series is Non Stationary

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
df_test = adfuller(index['Passengers'],autolag='AIC')
df_test

Output shows the follwoing : Test Statistic 0.81 p-value 0.99 #Lags Used 13 No. of Observation Used 130 followed by Critical Values at various percentages.

**For Data to be stationary p value should be < 0.05 and critical values should be close to Test Statistics. Therefore, ADCF test concludes that data is NOT Stationary**

Still y'all have a doubt whether Data is stationary or not? Don't worry. Just for your validation, I will conduct another test! That being the KPSS Test

In [ ]:
from statsmodels.tsa.stattools import kpss

In [ ]:
kpss_test = kpss(index['Passengers'],'ct')
kpss_test

Output shows the follwoing : Test Statistic 0.16 p-value 0.03 #Lags Used 14 followed by Critical Values at various percentages.

For Data to not be stationary p value should be < 0.05 and Null Hypothesis should be rejected. Therefore, KPSS test also concludes that data is NOT Stationary



# Transforming Data to Achieve Stationarity

In [ ]:
index_log = np.log(index)
ma_log = index_log.rolling(window=11).mean()
std_log = index_log.rolling(window=12).std()

In [ ]:
plt.title("Logarithmic Values vs MA vs Std")
plt.plot(index_log)
plt.plot(ma_log)
plt.plot(std_log)

This seems to be sligtly better than the precious case. So hand on there guys. We're getting close

In [ ]:
index_new = index_log - ma_log
index_new.dropna(inplace=True)

In [ ]:
ma_new = index_new.rolling(window=11).mean()
std_new = index_new.rolling(window=12).std()

In [ ]:
plt.title("New Index Values vs MA vs Std")
plt.plot(index_new)
plt.plot(ma_new)
plt.plot(std_new)

In [ ]:
df_test_new = adfuller(index_new['Passengers'],autolag='AIC')
df_test_new

After concluding the results from ADFC TEST, p-value has reduced from 0.99 to 0.019 AND critical values at 1%,5%,10% confidence intervals are pretty close to the Test Statistic. Thus,we can say that our given series is stationary.

# ARIMA MODEL CREATION

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(index_log , order = (2,1,2))
results_ARIMA = model.fit(disp=-1)

In [ ]:
predictions = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions.head()

Link for part 2 https://www.kaggle.com/harshsinha1234/time-series-arima-model-simplified

In [ ]:
model2 = ARIMA(index_log , order = (2,1,0))
results_ARIMA2 = model2.fit(disp=-1)
model3 = ARIMA(index_log , order = (0,1,2))
results_ARIMA3 = model3.fit(disp=-1)

In [ ]:
predictions2 = pd.Series(results_ARIMA2.fittedvalues, copy=True)
predictions2.head()

In [ ]:
predictions3 = pd.Series(results_ARIMA3.fittedvalues, copy=True)
predictions3.head()